In [5]:
from bs4 import BeautifulSoup
import requests
import pickle

import pandas as pd
import numpy as np

from multiprocessing.pool import ThreadPool as Pool

import warnings
warnings.filterwarnings('ignore')

In [10]:
url_haz = 'https://ingatlan.com/szukites/elado+haz?page='
last_page_haz = 2886

urls_hazak = []
for i in range(1, last_page_haz + 1):
    urls_hazak.append(url_haz + str(i))
    
url_lakas = 'https://ingatlan.com/szukites/elado+lakas?page='
last_page_lakas = 3107

urls_lakasok = []
for i in range(1, last_page_lakas + 1):
    urls_lakasok.append(url_lakas + str(i))

In [11]:
print(urls_hazak[-1])
print(urls_lakasok[-1])

https://ingatlan.com/szukites/elado+haz?page=2886
https://ingatlan.com/szukites/elado+lakas?page=3107


### For now concentrate only on houses

In [15]:
# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
headers = {'User-Agent': 'chrome gecko'}

In [27]:
def get_links(url):
    
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.text)
    talalatok = soup.find_all('div', class_ = 'listing__card')
    
    linkek = []
    
    for i in talalatok:
        linkek.append('https://ingatlan.com' + i.find('a', href = True)['href'])
        
    return linkek

Run for all pages

In [40]:
%%time

osszes_link = []

pool = Pool(4)

for page in urls_hazak:
    pool.apply_async(get_links, (page,), callback = osszes_link.extend)

pool.close()
pool.join()

Wall time: 18min 21s


For 100 it took 40 secs --> for 2886 it will take 2886 / 100 * 40 / 60 = 20 mins

In [41]:
len(osszes_link)

57720

In [42]:
with open('data/osszes_link.pkl', 'wb') as pkl:
    pickle.dump(list(set(osszes_link)), pkl)